In [2]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

cap = cv2.VideoCapture("short head biceps.mp4")

# Curl counter variables
counter = 0 
stage = None

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    #print(a,b,c)
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 
    

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5,model_complexity=1) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        image = cv2.resize(image,(960,640))
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            left_ankle_visibility=landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
            right_ankle_visibility=landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility
            if False:
                (left_ankle_visibility <0.70) or (right_ankle_visibility <0.70)
                print(left_ankle_visibility)
                cv2.putText(image, str("please go back"), 
                                        (200,50), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2, cv2.LINE_AA)
                          
            else:
                
                

                # Get coordinates for elbow
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
        
                #print(shoulder,elbow,wrist)
                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)

                # Visualize angle
                cv2.putText(image, str(angle), 
                               tuple(np.multiply(elbow, [960, 640]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA
                                    )

                # for knee 
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                #print(hip,knee,ankle)
                # Calculate angle
                knee_angle = calculate_angle(hip, knee, ankle)
                #print(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility)
                #print(knee_angle)
                # Visualize angle
                cv2.putText(image, str(knee_angle), 
                               tuple(np.multiply(knee, [image.shape[1], image.shape[0]]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA
                                    )
                #for hip angle
                shoulder= [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]

                hip_angle=calculate_angle(shoulder,hip,knee)
                shoulder_angle=calculate_angle(hip,shoulder,elbow)
                cv2.putText(image, str(shoulder_angle), 
                               tuple(np.multiply(shoulder, [960, 640]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA
                                    )
                print(shoulder_angle)
                cv2.putText(image, str(hip_angle), 
                               tuple(np.multiply(hip, [960, 640]).astype(int)), 
                               cv2.FONT_HERSHEY_SIMPLEX, 5, (0, 0, 255), 5, cv2.LINE_AA
                                    )
                if hip_angle <165:
                    cv2.putText(image, str("Stand straight"), 
                                (200,50), 
                                  cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,0), 2, cv2.LINE_AA)

                #print(hip_angle)
                # Curl counter logic
                #print(angle)
                if (angle > 110 and knee_angle>160 and shoulder_angle>70):
                    #print(angle)
                    stage = "down"
                else:
                    cv2.putText(image, 'Lift up properly', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                if angle < 90 and stage =='down' and knee_angle>160 and shoulder_angle>70:
                    stage="up"
                    counter +=1
                    #print(counter)
                # Render curl counter
                # Setup status box
                cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)

                # Rep data
                cv2.putText(image, 'REPS', (15,12), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
                cv2.putText(image, str(counter), 
                            (10,60), 
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

                # Stage data
        #         cv2.putText(image, 'STAGE', (65,12), 
        #                     cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        #         cv2.putText(image, stage, 
        #                     (60,60), 
        #                     cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)


                # Render detections
                mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                        mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                        mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                         )               

            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

           

        except:
            pass

    cap.release()
    cv2.destroyAllWindows()  

124.89560038879017
125.03148642913965
141.88560895884186
97.15321135657882
98.49240836396609
99.08287784993479
100.99652223227312
102.76262264317513
104.0363008623878
104.9198546178915
105.30972700920113
105.22093482110564
105.99876809267599
106.58365551542867
104.7258712510097
100.90094853430459
94.86700078084655
93.98803520166432
91.36220327494463
89.10704340023698
89.32462634437111
90.78880047343367
91.3196967148791
91.59095798329288
92.4719525679421
93.45074818980035
92.12422453015013
91.83784581360631
91.66490152184058
99.42962511713128
102.53642106725951
104.09934999419124
105.57872994187342
107.03096948269614
107.9353099714165
108.54564699946488
108.93080626685934
108.951253298478
108.12914454584802
107.62034316899003
107.23129477149574
106.55705570901125
105.73181263892586
105.2307732270843
104.47929597580381
103.17795991326003
101.2034191036772
100.43046728230604
100.89900317753099
102.47009522352405
102.82776879192602
102.7653002730041
102.34547161116838
103.01899938335342
10

101.83774258048005
102.88778214630312
102.54228549198274
102.4852034232894
102.76071457160822
102.98243006169378
103.10517016243108
103.23457435990969
103.12569767458079
102.88288656754978
102.41697777213399
102.12050513896277
101.89003640934328
101.55740425970049
101.79618057479517
101.15916993574723
100.88133256231143
101.35756069121649
101.22449236664771


error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
